<img src="./img/HWNI_logo.svg"/>

# Lab - Hypothesis Testing

In [ ]:
# makes our plots appear inside Jupyter
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

import util.utils as utils
import util.shared as shared

shared.format_plots()
shared.format_dataframes()

In this lab, we use simulated experiments to investigate the ideas covered in the
[tutorial on Hypothesis Testing](./Tutorial\ -\ Hypothesis\ Testing.ipynb).

The experiment we simulate is inspired by a 
[classic neuroscience experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1514868/)
performed by Adrian in the 1920s.

At the time, very little of what we now take for granted was known about the behavior of neurons. This early experiment demonstrated that the nerves exiting the cat's foot produced more action potentials when a weight was pressed onto the foot in a manner that depended on the weight.

<img src="./img/adrian_yngve_1926_apparatus.gif">

We'll model this experiment using the random number generators provided with NumPy. The function `run_experiment` below will produce a random spike count using the
[Poisson](https://en.wikipedia.org/wiki/Poisson_distribution)
distribution, which has been used to
[model neural spiking in response to stimuli](https://en.wikipedia.org/wiki/Linear-nonlinear-Poisson_cascade_model),
then compare it to the argument `critical_value`.
The result of the test is returned as number:
`0` if the alternative hypothesis is not accepted
and `1` if the alternative hypothesis is accepted.
If the random spike count is higher than the critical value,
the alternative hypothesis is accepted,
otherwise it is not accepted.

The average value of the random spike count is determined by a parameter.
This parameter is set by two of the arguments to `run_experiment`: `baserate`, which we can think of as setting the average number of spikes when the foot is not being stimulated, and `difference`, which corresponds to the change in this average when the foot is stimulated.

This is a very simple experiment and a very simple statistical test, but it will still allow us to see the major features of hypothesis testing discussed in the tutorial:
true positive rates, false positive rates, and false discovery rates.

In general hypothesis testing terms, the *test statistic* we are using is just the value of the data point that we observed. We might call the measurements from the unstimulated foot the *control measurements* and the measurement from the stimulated foot the *experimental measurement*.

In [ ]:
def run_experiment(baserate, difference, critical_value):
    """ run a single experiment where the number of spikes recorded from a 
        neuron firing with rate baserate+difference
        is compared to the critical_value.
        
        returns a 1 if the (randomly-generated) number of spikes
        is higher than the cutoff, otherwise returns a 0
        
        the number of spikes comes from a poisson distribution
    """
    stimulated_spikes = np.random.poisson(lam=baserate+difference)
    
    if stimulated_spikes > critical_value:
        accepted = 1
    else:
        accepted = 0
    
    return accepted

In [ ]:
def simulate_experiments(num_experiments, rate, effect_size, critical_value):
    """ simulate num_experiments experiments with run_experiment
        and return the result as a list.
        
        this list has 1s where the null hypothesis was rejected
        and 0s where the null hypothesis was not rejected
    """
    results = np.zeros(num_experiments)
    
    for experiment_idx in range(num_experiments):
        result = run_experiment(rate, effect_size, critical_value)
        results[experiment_idx] = result
    
    return results

## True and False Positive Rates

To start, we'll estimate the true and false positive rates.

#### Q1 In your own words, what are true and false positive rates? What's the difference between a false positive rate and a false discovery rate?

### False Positive Rate

We can estimate the false positive rate by simulating the results of our hypothesis test when the effect size, the difference in average spike counts between stimulated and unstimulated nerves, is 0.

#### Q2 What is the connection between the effect size being 0 and the false positive rate?

The results of the experiments are stored in a list called `nulltrue_results`. Each element is either a `1` or ` 0`, depending on whether the alternative hypothesis was accepted or not. Use this list to calculate the fraction of times that we got a (false) positive result. You could use a `for` loop or the `sum` function or the `np.mean` function.

In [ ]:
rate = 10
effect_size = 0

critical_value = 15
num_experiments = 1000

nulltrue_results = simulate_experiments(num_experiments, rate, effect_size,
                                       critical_value)


In [ ]:
# your code here

The false positive rate depends critically (ha!) on the critical value of our test. Remember that the critical value is the value of the test statistic (for us, this is just the observed spike count) above which you reject the null hypothesis.

#### Q3 Make a prediction: will increasing the critical value increase or decrease the false positive rate?

Test your prediction in the cell below.

In [ ]:
rate = 10
effect_size = 0

critical_value = 17
num_experiments = 1000

nulltrue_results_higher_crit_value = simulate_experiments(num_experiments, rate, effect_size,
                                       critical_value)

In [ ]:
# your code here

As can be seen above, changing the critical value changes the false positive rate. The two code cells below will allow us to examine this dependence in detail.

#### Q4 Make a prediction: what will the false positive rate be when the critical value is 0? what about when the critical value is very large?

The first cell below calculates the rate of false positives for experiments with a range of critical values and stores them in a list called `false_positive_rates`.

The cell beneath it will then plot the resulting false positive rates as a function of the critical value.

In [ ]:
false_positive_rates = []

rate = 10
effect_size = 0
num_experiments = 100000

critical_values = range(0,21)

for critical_value in critical_values:
    nulltrue_results = simulate_experiments(num_experiments, rate, 0,
                                       critical_value)
    
    false_positive_rates.append(np.mean(nulltrue_results))

In [ ]:
utils.plot_rates(critical_values, false_positive_rates, "False Positive Rate")

## True Positive Rate

We can estimate the true positive rate by simulating the results of our hypothesis test when the effect size is non-zero.

The cell below will produce a list, `nullfalse_results`, just like the `nulltrue_results` list from above. Use it to calculate the true positive rate.

In [ ]:
rate = 10
effect_size = 1

critical_value = 15
num_experiments = 10000

nullfalse_results = simulate_experiments(num_experiments, rate, effect_size,
                                       critical_value)

In [ ]:
# your code here

The true positive rate depends, just like the false positive rate, on the critical value.

#### Q4 Make a prediction: will increasing the critical value increase or decrease the true positive rate?

Test your prediction using the cells below.

In [ ]:
rate = 10
effect_size = 1

critical_value = 17
num_experiments = 1000

nullfalse_results = simulate_experiments(num_experiments, rate, effect_size,
                                       critical_value)

In [ ]:
# your code here

As above, we can simulate the results for a variety of critical values. The two cells below calculate and then plot the true positive rate with varying critical values. For comparison, the false positive rate is also plotted.

In [ ]:
true_positive_rates = []

rate = 10
effect_size = 1
num_experiments = 100000

critical_values = range(0,21)

for critical_value in critical_values:
    nullfalse_results = simulate_experiments(num_experiments, rate, effect_size,
                                       critical_value)
    
    true_positive_rates.append(np.mean(nullfalse_results))

In [ ]:
utils.plot_true_and_false_positive_rates(critical_values,
                                         false_positive_rates,
                                         true_positive_rates)

#### Q5 Based on this chart, what would you select as your critical value? There is no single correct answer. Explain your reasoning.

### TPR as a Function of Critical Value and Effect Size

The true positive rate also depends on the actual size of the effect you're looking for.

The code cells below will produce an interactive 3-d plot of the true positive rate as a function of both the effect size and the critical value. Left-clicking and dragging allows you to rotate the plot, while right-clicking and dragging allows you to zoom.

These interactive features are enabled by the `%matplotlib notebook` magic. **If the plot doesn't show up the first time you run the cell, run it again**.

Once you're done with this plot, run the cell underneath it, containing the `%matplotlib inline magic`, to return plots to their normal, non-interactive forms. You can also run the first code cell (the one with all of the imports).

In [ ]:
baserate = 10

num_experiments = 10000

critical_values = list(range(5,21))
effect_sizes = list(range(1,16))

TPR = utils.estimate_TPR(baserate, effect_sizes, critical_values, simulate_experiments,
                         num_experiments=num_experiments)

In [ ]:
%matplotlib notebook
shared.format_plots()

utils.plot_TPR(critical_values, effect_sizes, TPR)

In [ ]:
%matplotlib inline
shared.format_plots()

#### Q6 Explain the shape of this graph: why does the rate go down when you decrease the effect size and hold the critical value constant? why are the extreme values -- closest to 1 (blue) and closest to 0 (red) -- located where they are?

#### Q7 Why can't we calculate a true positive rate for the case where `effect_size == 0`?

## $p$-Value

If we have a good model of our null hypothesis, then we do more than just say whether we reject or fail to reject the null. If we can determine the probability that, if the null hypothesis were true, we'd observe what we observed, then we can express the degree to which our results support the null hypothesis. This probability is called the $p$-value.

In many familiar statistical tests, this model is *parametric* -- we assume some form for the distribution of the data, and therefore the test statistic, under the null hypothesis and then, possibly after measuring some data generated according to the null hypothesis and inferring those parameters, we use the mathematical form of that distribution to calculate the $p$-value.

When we don't know what form to assume the null distribution has, we can use a *nonparametric* method instead. In a non-parametric method, we instead use the data measurements to directly estimate the shape of the null distribution of the test statistic. Since our test statistic is just the number of spikes that we measure in a single experiment, that means we just need to collect a large number of spike counts distributed according to the null hypothesis.

So in our case, we estimate the $p$-value by repeatedly measuring the spike counts from a neuron in an unstimulated cat's foot and then calculating how often that spike count is higher than the spike count from the neuron when the foot is stimulated.

The cell below will simulate measuring the spike counts from an unstimulated foot `number_null_measurements` times, along with a single measurement of the spike count from a stimulated foot.

It then plots a histogram of the spike counts from the unstimulated foot along with a single tick mark indicating the number of spikes measured from the stimulated foot. The bins are selected so that each bin contains a single value of the spike count.

In [ ]:
number_null_measurements = 10000

difference = 5
baserate = 10

null_spikes = np.random.poisson(lam=baserate,size=10000)
experiment_spikes = np.random.poisson(lam=baserate+difference)

utils.plot_null_and_result(null_spikes, experiment_spikes)

#### Q8 What is the relationship between this histogram and the null distribution of our test statistic?

A rough guess for the $p$-value can be read directly off the chart above.

#### Q9 Explain how you'd do this.

To calculate the $p$-value precisely, we need the actual heights for this histogram.

#### Q10 How would we calculate the $p$-value from this information?

The cell below uses the function `np.histogram` to collect these values and store them in the list `probabilities`. This list acts like a probability mass function: accessing the `i`th element of the list with `probabilities[i]` tells you the fraction of experiments in which `i` spikes occurred.

In [ ]:
bins = utils.get_bins(null_spikes,experiment_spikes)

probabilities, bin_edges = np.histogram(null_spikes, bins=bins, normed=True)

Calculate the $p$-value in the cell below.

*Hint:* things you might use include: `for` loops, `if` statements amd the `np.sum` function.
Also: `list[i:]` will return all the values in a list from the `i`th element to the end of the list.

In [ ]:
# your code here

Simulate a different outcome for the experiment by running the cell below to collect a different random value for `stimulated_spikes`.

In [ ]:
experiment_spikes = np.random.poisson(lam=baserate+difference)
utils.plot_null_and_result(null_spikes, experiment_spikes)

#### Q11 Re-calculate the $p$-value. Is it the same or different? Why is this?

In addition to the probability distribution, the *cumulative distribution function* can be used to represent the chance that a random variable, like a test statistic, takes on some value. The cumulative distribution function, also known as the CDF, takes in a value and returns the probability that the random variable is *less than or equal to* that value. Put another way, it tells you how much probability has *accumulated* as the value of the random variable increases from some minimum up to the current value.

The cell below calculates and plots the cumulative distribution function for the null distribution of our test statistic.

#### Q12 How might this information be used to calculate the $p$-value?

In [ ]:
CDF, new_edges = utils.make_CDF(probabilities, bin_edges)
utils.plot_CDF(CDF, new_edges)

Recall that the $p$-value is the probability, under the null hypothesis, that the test statistic takes on a value that is at least as extreme as the measured value.

#### Q13 Based on this, can you determine the chance, when the null hypothesis is true, that the $p$-value is less than 0.5? 0.05? 0?

Thanks to our simulation framework, we can answer this question directly. The cell below will compute the $p$-value for a large number of experiments and then present the histogram of the results.

In [ ]:
baserate = 1000
null_spikes = np.random.poisson(lam=baserate,size=100000)
bins = utils.get_bins(null_spikes,baserate+baserate)

probabilities, bin_edges = np.histogram(null_spikes, bins=bins, normed=True)

In [ ]:
num_experiments = 10000
effect_size = 1

CDF, new_edges = utils.make_CDF(probabilities, bin_edges)

ps = [1 - CDF(experiment_spikes) for experiment_spikes 
                        in np.random.poisson(lam=baserate+effect_size,size=num_experiments)]

In [ ]:
utils.plot_p_distribution(ps)

#### Q14 What shape does this distribution have?

Run the code cell above with the `effect_size` set to `10`.

#### Q15 Now what shape does the distribution have?

This information about the shape of the distribution of $p$-values can be used to calculate false discovery rates, but only if we run a variety of very similar experiments. For more, see this
[Points of Significance article](http://www.nature.com/nmeth/journal/v11/n4/full/nmeth.2900.html).